In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint
import time

In [2]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = False):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [3]:
ProdTestconfigLag0Target1 = ConfigElements(0,[ ("A",["Semana","Ruta_SAK"],["count","count"])], "lag0tar1", False)
print  ProdTestconfigLag0Target1.__dict__

{'nameAndGroups': [('A', ['Semana', 'Ruta_SAK'], ['count', 'count'])], 'lag': 0, 'targetVariable': 'lag0tar1', 'deleteColumns': False}


In [4]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.reset_index(inplace=True)
        data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana.values == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana.values == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  np.apply_along_axis(func,0,FE.train[column].values).astype(localType)
                #np.apply_along_axis(lambda x: x+1,0,FE.train["Semana"]).astype("int32")
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  np.apply_along_axis(func,0,FE.test1[column].values).astype(localType)
                self.test2.loc[:,column] =  np.apply_along_axis(func,0,FE.test2[column].values).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self): 
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
         #   self.train.loc[:,'Semana'] < 10].mean().astype("float32")
        self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].values[
        (self.train.loc[:,'Semana'].values < self.ValidationStart).values].mean().astype("float32")
        #self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        #self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        #display(self.train)
        #del DemandaMeanWithoutLag
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train[self.train['Semana'].values <= (self.ValidationEnd - config.lag)]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].values + config.lag
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].copy().groupby(groups).agg(aggregate[0])
                gc.collect()
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            display(self.test1)
            display(self.test2)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable].values), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable].values)
                    , name].values
                self.test1.loc[pd.isnull(self.test1[config.targetVariable].values), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable].values),
                    name].values
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable].values), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable].values)
                        , name].values
                    
                count = self.test1[config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train)   
        display(self.test1)   
        display(self.test2)
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self,trainOrTestOrBoth):
        self.train = self.train[self.train['Semana'].values >= (3 + self.maxLag)]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
        #BAD PERFORMANCE!!
    #Split train data to train and test1 and test2 (validation)
    #def SplitTrainToTestUsingValidationStart(self):
     #   boolCondition = self.train.Semana == self.ValidationStart
      #  self.test1 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
       # boolCondition = self.train.Semana == self.ValidationEnd
       # self.test2 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
      #  del boolCondition
      #  gc.collect()
    
    #Reaches 3x memory from train, because of test1, test2 and train itself at the end.. GC fixed in the end..
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana.values == self.ValidationStart
        self.test1 = self.train[boolCondition]
        boolCondition = self.train.Semana.values == self.ValidationEnd
        self.test2 = self.train[boolCondition]
        FE.train = FE.train[ FE.train.Semana.values < FE.ValidationStart ]
        del boolCondition
        gc.collect()

In [5]:
parameterDict =       {"ValidationStart":4, 
 "ValidationEnd":5,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1.h5",
    "testHdfPath2":"../../input/test2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1.h5', 'ValidationEnd': 5, 'testHdfPath2': '../../input/test2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 4, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5'}


In [6]:
#FE.ReadHdf('train')
FE.ReadFirstNRowsOfACsv(23000000,'train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23000000 entries, 0 to 22999999
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 658.0 MB
None


In [7]:
now = time.time()
FE.SplitTrainToTestUsingValidationStart()
print time.time() - now

1.63946294785


In [9]:
now = time.time()

tempData = FE.train.loc[FE.train.loc[:,'Semana'].values <= (FE.ValidationEnd - 0)]
print time.time() - now

0.686796188354


In [10]:
now = time.time()
tempData = tempData[['Semana','Canal_ID', 'Demanda_uni_equil'
                    ]].groupby(['Semana','Canal_ID']).agg(['count'])
print time.time() - now
tempData.head(5)


1.43593502045


Demanda_uni_equil
                            count
Semana Canal_ID                  
3      1                 10187879
       2                   120523
       4                   550739
       5                    21071
       6                    37410

In [11]:
tempData.reset_index(inplace=True)

In [13]:
tempData.drop("Semana",axis=1, inplace=True)

/Users/berkerkozan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  if __name__ == '__main__':


In [14]:
tempData.head(5)

,Canal_ID,Demanda_uni_equil
,,count
0,1,10187879
1,2,120523
2,4,550739
3,5,21071
4,6,37410


In [10]:
gc.collect()

145

In [11]:
tempData.reset_index(inplace=True)

In [12]:
tempData.drop("Semana",axis=1, inplace=True)

/Users/berkerkozan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  if __name__ == '__main__':


In [13]:
display(FE.train.head(2))
display(tempData.head(2))
tempData.info()
FE.train.info()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


,Canal_ID,Demanda_uni_equil
,,count
0,1,10187879
1,2,120523


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
(Canal_ID, )                  9 non-null int64
(Demanda_uni_equil, count)    9 non-null int64
dtypes: int64(2)
memory usage: 216.0 bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11165207 entries, 0 to 11165206
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 404.6 MB


In [26]:
FE.train =  FE.train.merge( tempData, left_on=["Canal_ID"],right_index=True, how='left', sort=False,copy=False)

In [24]:
FE.train.head(10)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,"(Canal_ID, )","(Demanda_uni_equil, count)"
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3,9.0,53.0
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4,9.0,53.0
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4,9.0,53.0
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4,9.0,53.0
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3,9.0,53.0
5,3,1110,7,3301,15766,1250,5,38.200001,0,0.0,5,9.0,53.0
6,3,1110,7,3301,15766,1309,3,20.280001,0,0.0,3,9.0,53.0
7,3,1110,7,3301,15766,3894,6,56.099998,0,0.0,6,9.0,53.0
8,3,1110,7,3301,15766,4085,4,24.600000,0,0.0,4,9.0,53.0
9,3,1110,7,3301,15766,5310,6,31.680000,0,0.0,6,9.0,53.0


In [28]:
FE.train.head(10)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,"('Canal_ID', '')_x","('Demanda_uni_equil', 'count')_x","(Canal_ID, )","(Demanda_uni_equil, count)"
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3,9.0,53.0,9.0,53.0
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4,9.0,53.0,9.0,53.0
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4,9.0,53.0,9.0,53.0
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4,9.0,53.0,9.0,53.0
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3,9.0,53.0,9.0,53.0
5,3,1110,7,3301,15766,1250,5,38.200001,0,0.0,5,9.0,53.0,9.0,53.0
6,3,1110,7,3301,15766,1309,3,20.280001,0,0.0,3,9.0,53.0,9.0,53.0
7,3,1110,7,3301,15766,3894,6,56.099998,0,0.0,6,9.0,53.0,9.0,53.0
8,3,1110,7,3301,15766,4085,4,24.600000,0,0.0,4,9.0,53.0,9.0,53.0
9,3,1110,7,3301,15766,5310,6,31.680000,0,0.0,6,9.0,53.0,9.0,53.0


In [18]:
FE.train = FE.train.merge( tempData, left_on=["Semana","Canal_ID"],
    right_index=True, how='left', sort=False,copy=False)

In [27]:
gc.collect()

113

In [ ]:
del grouped3,tempData
gc.collect()

In [15]:

print time.time() - now
grouped.head(5)

166.75


Demanda_uni_equil
                            count
Semana Canal_ID                  
3      1                 10187879
       2                   120523
       4                   550739
       5                    21071
       6                    37410

In [21]:
gc.collect()

720

In [24]:
%timeit tempData['Semana'].values.copy() + 1

100 loops, best of 3: 11.3 ms per loop


In [25]:
%timeit tempData['Semana'].values + 1

100 loops, best of 3: 8.25 ms per loop


In [82]:
testDF = pd.DataFrame({"a":[np.nan,2,3,np.nan,5],"b":[6,7,8,9,10]})

In [63]:
#self.train.loc[pd.isnull(self.train[config.targetVariable]), 
 #                   config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]

In [64]:
%timeit testDF.loc[pd.isnull(testDF["a"]), "a"] = testDF.loc[pd.isnull(testDF["a"]), "b"]

1000 loops, best of 3: 1.09 ms per loop


In [79]:
testDF

,a,b
0,6.0,6
1,2.0,7
2,3.0,8
3,9.0,9
4,5.0,10


In [77]:
%timeit testDF.loc[pd.isnull(testDF["a"].values), "a"] = testDF.loc[pd.isnull(testDF["a"].values), "b"].values

1000 loops, best of 3: 843 µs per loop


In [93]:
%timeit testDF["a"].isnull().sum()

10000 loops, best of 3: 124 µs per loop


In [92]:
%timeit np.isnan(testDF["a"]).sum()

10000 loops, best of 3: 121 µs per loop


In [72]:
type(pd.isnull(testDF["a"].values))

numpy.ndarray

In [74]:
type(testDF["a"].values)

numpy.ndarray

In [13]:
tempData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11165207 entries, 0 to 11165206
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 404.6 MB


In [11]:
del grouped3,tempData
gc.collect()

12

In [19]:
grouped.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 3 columns):
(Semana, )                    551 non-null int64
(Agencia_ID, )                551 non-null int64
(Demanda_uni_equil, count)    551 non-null int64
dtypes: int64(3)
memory usage: 13.0 KB


In [10]:
del grouped3
gc.collect()

152

In [13]:
del tempData
gc.collect()

18

In [18]:
tempData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11165207 entries, 0 to 11165206
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 404.6 MB


In [131]:
FE.train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,berker
0,16,1110,7,3301,15766,1212,3,25.139999,0,0.0,3,NaN
1,16,1110,7,3301,15766,1216,4,33.520000,0,0.0,4,NaN
2,16,1110,7,3301,15766,1238,4,39.320000,0,0.0,4,NaN
3,16,1110,7,3301,15766,1240,4,33.520000,0,0.0,4,NaN
4,16,1110,7,3301,15766,1242,3,22.920000,0,0.0,3,NaN
5,16,1110,7,3301,15766,1250,5,38.200001,0,0.0,5,NaN
6,16,1110,7,3301,15766,1309,3,20.280001,0,0.0,3,NaN
7,16,1110,7,3301,15766,3894,6,56.099998,0,0.0,6,NaN
8,16,1110,7,3301,15766,4085,4,24.600000,0,0.0,4,NaN
9,16,1110,7,3301,15766,5310,6,31.680000,0,0.0,6,NaN


In [8]:
FE.train.info()
FE.test1.info()
FE.test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11165207 entries, 0 to 11165206
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 404.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11009593 entries, 11165207 to 22174799
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 399.0 MB

In [10]:
FE.train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.6 GB


In [10]:
FE.train.drop( np.array(FE.train.loc[boolCondition].index), axis=0,inplace=True)

KeyboardInterrupt: 

In [15]:
del boolCondition

In [11]:
gc.collect()

9

In [ ]:
FE.SplitTrainToTestUsingValidationStart()

In [12]:
FE.train.info()
FE.test1.info()
FE.test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11165207 entries, 0 to 11165206
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 404.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11009593 entries, 11165207 to 22174799
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 399.0 MB

In [ ]:
display(FE.test1.head(2))
display(FE.test2.head(2))
display(FE.train.head(2))

In [ ]:
FE.AddConfigurableFeaturesToTrain(ProdTestconfigLag0Target1)